In [2]:
import torch
from matplotlib import pyplot as plt

#torch.set_default_device('cuda')


t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4])

In [3]:
def model(t_u, w, b):
    return w * t_u + b

In [10]:
def loss_fn(t_p, t_c):
    return torch.mean((t_p - t_c) ** 2)

In [5]:
w = torch.ones(1)
b = torch.zeros(1)

In [6]:
t_p = model(t_u, w, b)
t_p

tensor([35.7000, 55.9000, 58.2000, 81.9000, 56.3000, 48.9000, 33.9000, 21.8000,
        48.4000, 60.4000, 68.4000])

In [12]:
loss_fn(t_p, t_c)

tensor(1763.8848)

In [14]:
params = torch.tensor([1., 0.], requires_grad=True)
params.grad is None

True

In [15]:
loss = loss_fn(model(t_u, *params), t_c)
loss

tensor(1763.8848, grad_fn=<MeanBackward0>)

In [18]:
loss.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [20]:
params.grad

tensor([4517.2969,   82.6000])

In [27]:
def training_loop(n_epochs, lr, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:
            params.grad.zero_()
        
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        loss.backward()

        params = (params - lr * params.grad).detach().requires_grad_()
        if epoch % 50 == 0:
            print(f"Current Epoch: {epoch},\tLoss: {float(loss)}")
    return params        

In [25]:
t_un = (t_u - t_u.mean()) / t_u.std()

In [31]:
training_loop(1000, 1e-2, torch.tensor([1., 0.], requires_grad=True), t_un, t_c)

Current Epoch: 50,	Loss: 27.870668411254883
Current Epoch: 100,	Loss: 6.498063564300537
Current Epoch: 150,	Loss: 3.443054437637329
Current Epoch: 200,	Loss: 3.0026862621307373
Current Epoch: 250,	Loss: 2.938664436340332
Current Epoch: 300,	Loss: 2.9292776584625244
Current Epoch: 350,	Loss: 2.9278886318206787
Current Epoch: 400,	Loss: 2.9276816844940186
Current Epoch: 450,	Loss: 2.9276506900787354
Current Epoch: 500,	Loss: 2.9276463985443115
Current Epoch: 550,	Loss: 2.927645683288574
Current Epoch: 600,	Loss: 2.927644968032837
Current Epoch: 650,	Loss: 2.9276459217071533
Current Epoch: 700,	Loss: 2.927645206451416
Current Epoch: 750,	Loss: 2.927645206451416
Current Epoch: 800,	Loss: 2.927645206451416
Current Epoch: 850,	Loss: 2.927645206451416
Current Epoch: 900,	Loss: 2.927645206451416
Current Epoch: 950,	Loss: 2.927645206451416
Current Epoch: 1000,	Loss: 2.927645206451416


tensor([ 9.0349, 10.5000], requires_grad=True)

In [32]:
from torch import optim
dir(optim)

['ASGD',
 'Adadelta',
 'Adagrad',
 'Adam',
 'AdamW',
 'Adamax',
 'LBFGS',
 'NAdam',
 'Optimizer',
 'RAdam',
 'RMSprop',
 'Rprop',
 'SGD',
 'SparseAdam',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_functional',
 '_multi_tensor',
 'lr_scheduler',
 'swa_utils']

In [33]:
optimizer = optim.SGD([params], lr=1e-3)

In [ ]:
def n_train(n_epochs, optimizer, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 50 == 0:
            print(f"Epoch: {epoch}, Loss: {float(loss)}")
    
    return params